<a href="https://colab.research.google.com/github/jsvon/HUGGINGFACE-LLM-course/blob/main/Fine_tuning_masked_llm_VON_VERSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [2]:
!git config --global user.email "jumavonne10@gmail.com"
!git config --global user.name "Juma Vonne"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
#Checking the number of parameters
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"'Distilbert number of parameters: {round(distilbert_num_parameters)}M'")
print   (f"'BERT number of parameters: 110M'")

'Distilbert number of parameters: 67M'
'BERT number of parameters: 110M'


In [6]:
text = "This is a great [MASK]"

In [7]:
#Distilbert tokenizer to produce inputs for the model
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# pick the [MASK] candidates with the higher logits

top_5_tokens = torch.topk(mask_token_logits, 8, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")


'>>> This is a great !'
'>>> This is a great .'
'>>> This is a great deal'
'>>> This is a great adventure'
'>>> This is a great ;'
'>>> This is a great ?'
'>>> This is a great success'
'>>> This is a great mystery'


**GETTING THE DATASET**

In [9]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [10]:
sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")
    print(f"'>>> Label: {row['label']}'")


'>>> Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'
'>>> Label: 1'

'>>> Review: This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stu

**PREPROCESSING THE DATA**

In [11]:
#Tokenizing the corpus without truncating
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

  #Using batched = TRUE to activate fast multithreading
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
  )

tokenized_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [12]:
#Checking model max length, helps determine how big chunks can be
tokenizer.model_max_length

512

In [13]:
#smaller chunk_size to fit memory
chunk_size = 128

In [14]:
#taking a few reviews from the tokenized training sets and printing number tokens per review
#Slicing produces a list of lists for each featuer
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 363'
'>>> Review 1 length: 304'
'>>> Review 2 length: 133'


In [15]:
#concatenating the examples with a simple dictionary comprehension
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}

total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 800'


In [16]:
#splitting the concatenated reviews into chunks of the given chunk_size
#above is done by iterating over the features in cocatenated_examples
#the last chunk if smaller can be dropped or padded
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 32'


In [17]:
#dropping the smaller chunks

def group_texts(examples):
  #concatenate all texts
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  #compute the length of the concatenated texts
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  #we drop the last chunk if its smaller than chunk_size
  total_length = (total_length // chunk_size) * chunk_size
  #split by chunks of max_len
  result = {
      k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
      for k, t in concatenated_examples.items()
  }

  #creating a new labels column
  result["labels"] = result["input_ids"].copy()
  return result


In [18]:
#applying group_texts() to tokenized datasets using Dataset.map()
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [19]:
#checking CLS and SEP
tokenizer.decode(lm_datasets["train"][11]["input_ids"])

'to see that " naughty sex film " that " revolutionized the film industry "... < br / > < br / > yeesh, avoid like the plague.. or if you must see it - rent the video and fast forward to the " dirty " parts, just to get it over with. < br / > < br / > [SEP] [CLS] i would put this at the top of my list of films in the category of unwatchable trash! there are films that are bad, but the worst kind are the ones that are unwatchable but you are suppose to like them because they are supposed to be good for'

In [20]:
tokenizer.decode(lm_datasets["train"][11]["labels"])

'to see that " naughty sex film " that " revolutionized the film industry "... < br / > < br / > yeesh, avoid like the plague.. or if you must see it - rent the video and fast forward to the " dirty " parts, just to get it over with. < br / > < br / > [SEP] [CLS] i would put this at the top of my list of films in the category of unwatchable trash! there are films that are bad, but the worst kind are the ones that are unwatchable but you are suppose to like them because they are supposed to be good for'

**FINETUNING DistilBert WITH TRAINER API**

Transformers comes prepared with a dedicated *DataCollatorForLanguageModeling* for just this task

We just have to pass it the tokenizer and an *mlm_probability* argument that specifies what fraction of the tokens to mask. We’ll pick 15%, which is the amount used for BERT and a common choice in the literature:

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [22]:
#collator expects a list of dicts, each dict represents a single chunk of contigous text
#word_ids are removed as they are not expected for this data
#Result is musk token being randomly inserted at various locations
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented i am curious - yellow from my video store because [MASK] all the controversy that surrounded [MASK] when it was first released in 1967. i also heard that at first it was seized by u. s. customs if it ever tried to enter this country, evangelist being a fan of films considered [MASK] controversial " i [MASK] [MASK] to see [MASK] for myself. < br / > < [MASK] [MASK] > the plot is centered around a young swedish drama student named lena who wants to learn everything she can about [MASK]. in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such'

'>>> as the vietnam war and race issues in the united states. in [MASK] asking politicians and ordinary denizens of stockholm [MASK] [MASK] [MASK] on politics, she has [MASK] with her drama [MASK], classmates, and married men. < br [MASK] > < br / > what kills [MASK] about i am curious - yellow is that 40 years ago, crushing was con

Random masking makes evaluation metrics not be deterministic when using the trainer

In [23]:
#trying with tokenizer.convert_ids_to_tokens()

samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.convert_ids_to_tokens(chunk)}'")



'>>> ['[CLS]', 'i', 'rented', 'i', 'am', 'curious', '-', 'yellow', 'from', 'my', 'video', 'store', 'because', 'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first', 'released', 'in', '1967', '.', '[MASK]', 'also', 'heard', 'that', '[MASK]', '[MASK]', 'it', 'was', 'seized', 'by', 'u', '.', 's', '.', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', 'this', 'country', ',', 'therefore', '[MASK]', 'a', 'fan', 'of', 'films', 'considered', '"', 'controversial', '[MASK]', '[MASK]', 'really', 'had', 'to', '[MASK]', 'this', 'for', 'myself', '.', '[MASK]', 'br', '[MASK]', '>', '<', 'br', '/', '>', 'the', 'plot', 'belgrade', 'centered', 'around', 'a', 'young', 'swedish', 'drama', 'student', 'named', 'lena', 'who', 'wants', 'to', 'learn', 'everything', 'she', 'can', 'about', 'life', '.', 'in', 'particular', 'she', 'wants', 'to', 'focus', 'her', 'attention', '##s', '[MASK]', 'making', 'some', 'sort', 'of', 'documentary', 'on', 'what', 'the', 'average', 's

In [24]:
#Data_collator is a function that takes a list of samples and converts themto a batch
# labels are all -100 except for the ones coreesponding to mask words

import collections
import numpy as np

from transformers import default_data_collator

wwm_probability  = 0.2

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        #create a map between word and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                  current_word = word_id
                  current_word_index += 1
                mapping[current_word_index].append(idx)

        #randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return default_data_collator(features)

In [25]:
samples = [lm_datasets["train"][i] for i in range (2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] i rented [MASK] am [MASK] - yellow from my video store because of all the [MASK] that surrounded it when it [MASK] first [MASK] in [MASK]. i also heard that [MASK] [MASK] it was [MASK] by u. s. [MASK] if [MASK] ever tried to enter this [MASK], therefore being a fan of films [MASK] " controversial " i really [MASK] to [MASK] [MASK] [MASK] [MASK]. < br / [MASK] < br [MASK] > the plot [MASK] centered [MASK] a young [MASK] drama student named [MASK] who wants to learn [MASK] she can about life. in particular she wants to focus [MASK] attentions to making some sort of documentary on what the average [MASK] [MASK] thought about certain political issues such'

'>>> as the vietnam war and race issues in the united states. in [MASK] asking [MASK] and [MASK] denizens of [MASK] about their [MASK] on politics, she has sex with her [MASK] teacher, classmates, and married men. < [MASK] / > [MASK] br / > [MASK] kills me [MASK] [MASK] [MASK] curious [MASK] yellow is [MASK] 40 years ago [MA

In [26]:
#downsampling training size for GPU purposes
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [28]:
#Training arguements
from transformers import TrainingArguments

batch_size = 64

# showing the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-imdb",
    overwrite_output_dir=True,
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
)

In [29]:
#Reinstatiating the trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
)

In [30]:
#Evaluation strategy for this is to use perplexity unlike metrics in token classification
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity starting model: {math.exp(eval_results['eval_loss']):.2f}")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jumavonne10 (jumavonne10-kabarak-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


>>> Perplexity starting model: 21.94


In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,2.518100,2.435716,0.001600
2,2.494200,2.382571,0.001600
3,2.468600,2.396489,0.001600


TrainOutput(global_step=471, training_loss=2.4943259820310426, metrics={'train_runtime': 157.9861, 'train_samples_per_second': 189.89, 'train_steps_per_second': 2.981, 'total_flos': 994208670720000.0, 'train_loss': 2.4943259820310426, 'epoch': 3.0})

In [34]:
eval_results = trainer.evaluate()
print(f">>> Perplexity final model: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity final model: 10.66


In [ ]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ed-imdb/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...295510.513cbd0764c5.528.0: 100%|##########| 7.21kB / 7.21kB            

  ...295743.513cbd0764c5.528.1: 100%|##########| 7.62kB / 7.62kB            

  ...ed-imdb/model.safetensors:  12%|#2        | 33.5MB /  268MB            

  ...296259.513cbd0764c5.528.2: 100%|##########|   425B /   425B            

CommitInfo(commit_url='https://huggingface.co/jvon65434/distilbert-base-uncased-finetuned-imdb/commit/30ea78fc6a36f4f939ad0ef3c0104b0cbb76d21f', commit_message='End of training', commit_description='', oid='30ea78fc6a36f4f939ad0ef3c0104b0cbb76d21f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jvon65434/distilbert-base-uncased-finetuned-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='jvon65434/distilbert-base-uncased-finetuned-imdb'), pr_revision=None, pr_num=None)

***Fine-tuning DistilBert with  🤗  Accelerate***

In [35]:
#DataCollatorForLanguageModeling  applies random masking with each evaluation
# to eliminate this randomness in perplexity scores, application of masking on the whole test set
# using the default data collator in  🤗 Transformers to collect batches during evaluation

def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [36]:
downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [37]:
#setting up the dataloaders
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [38]:
#loading a fresh version of the pretrained model
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [39]:
#specifying the optimizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [40]:
#instatiating the accelerator
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [41]:
#Specifying learning rate scheduler
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [42]:
#Creating this model repo on the hub
from huggingface_hub import get_full_repo_name

model_name = "distilbert-base-uncased-finetuned-imdb-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

'jvon65434/distilbert-base-uncased-finetuned-imdb-accelerate'

In [43]:
from huggingface_hub import Repository

output_dir = model_name
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/jvon65434/distilbert-base-uncased-finetuned-imdb-accelerate into local empty directory.


In [44]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/471 [00:00<?, ?it/s]

>>> Epoch 0: Perplexity: 11.653398771449478
>>> Epoch 1: Perplexity: 11.149156447407861
>>> Epoch 2: Perplexity: 10.979039856759632


In [49]:
#using the finetuned model
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="jvon65434/distilbert-base-uncased-finetuned-imdb-accelerate"
)

config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [50]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> this is a great!
>>> this is a great.
>>> this is a great film
>>> this is a great movie
>>> this is a great adventure
